# IRF - Uppsala Python Workshop: Snakes in Space 🐍
author: Louis Richard
e-mail: louisr@irfu.se
date: 29/02/2024

## Python for data analysis:
- Numpy (https://numpy.org/doc/stable/index.html, https://numpy.org/doc/stable/user/numpy-for-matlab-users.html)
- Matplotlib (https://matplotlib.org/stable/tutorials/pyplot.html)
- Scipy (https://docs.scipy.org/doc/scipy/index.html)
- Jupyter (https://jupyter.org/)
These three modules are not included by default with python and will need to be installed somehow.  `pip install numpy matplotlib scipy jupyterlab`, for example

## Numpy

In [ ]:
import numpy as np  # convention, shorter, saves writing numpy all the time

### Create an array
numpy provides n-dimensional arrays, implemented at low-level for efficiency

In [ ]:
my_array = np.random.random((3, 3))

# We can do maths directly with them
something_new = 5.0 * np.sin(my_array)**2.

# There are many ways to create them.
another_array = np.zeros_like(my_array)
a_range = np.linspace(0., 1., 64)
another_range = np.arange(-1., 1., 0.1)

my_array

In [ ]:
my_array.astype(np.float32)

In [ ]:
my_array.astype(np.float64) 

In [ ]:
# Numpy arrays, like everything else, are objects, and have methods and attributes
print(f"{my_array.std() = }")
print(f"Number of dimensions = {my_array.ndim}, shape = {my_array.shape}")

**Broadcasting** rules applied to numerical operations on ndarrays. Length-1 dimensions get expanded in e.g. element-by-element multiplication.

https://numpy.org/doc/stable/user/basics.broadcasting.html

In [ ]:


# You might want to check the shapes of these things to understand what is going on. 
id_matrix = np.identity(3)
scalar = 3. + 1.0j
vec = np.random.randn(8).reshape((8,1,1))

(my_array * vec[:,...] * id_matrix + scalar).shape

### Array slicing
To extract / set parts of an array, re-order, re-shape.  But, such operations (generally) create only a "view" of the original array, not a deep copy!


In [ ]:
# What happens here?
new_array = my_array 
new_array = new_array[::-1, :]
new_array[1, 1] = 50
print(new_array)
print(my_array)

In [ ]:
copy_array = my_array.copy() 
copy_array = copy_array[::-1, :]
copy_array[1, 1] = 10
print(copy_array, my_array)


In [ ]:
# Using "where" to search and index 
my_array = np.random.random((3, 3))
inx = np.where(my_array > 0.5)
my_array[inx] = np.nan
print(my_array)

In [ ]:
inx = np.isnan(my_array)
my_array[inx] = -1e99
my_array

### Operating on arrays

In [ ]:
theta = np.deg2rad(30)
vec = np.array([1, 1, 1])
mat = np.array([
    [np.cos(theta), np.sin(theta), 0], 
    [-np.sin(theta), np.cos(theta), 0], 
    [0, 0, 1]])

# Applies the broadcasting rules
mat * vec

In [ ]:
np.matmul(mat, vec)

In [ ]:
r_mat = np.stack([np.array([[np.cos(theta), np.sin(theta), 0], [-np.sin(theta), np.cos(theta), 0], [0, 0, 1]]) for theta in np.random.rand(100)])

In [ ]:
b = [1, 0, 0] * np.ones((100, 3))

In [ ]:
m = np.random.rand(100, 3, 4, 5, 6, 8)
print(m[0, :, :, :, :, :].shape)
print(m[0].shape)
print(m[0, ...].shape)
print(m[..., 0].shape)
print(m[..., 0, :, :].shape)

In [ ]:
np.matmul(r_mat[0, ...], b[0, :])

In [ ]:
r_mat_b = np.matmul(r_mat, b[..., None])
print(r_mat.shape, b.shape, r_mat_b.shape)

## datetime64
Python has the `datetime` module ("batteries included", remember?).  

Within `numpy`, there is an implementation that can be used in array operations, `datetime64`.  Internally, dates are stored as a signed integer number of ticks of \[precision\], e.g. nanoseconds (giving about ±300 years of useable range from 1970).

Unless you really know what you're doing, you probably want to try and stick to using nanosecond precision.

Differences between `datetime64`s are implemnted as `timedelta64`s.


In [ ]:
# A time string
"2017-08-07T00:00:00.000000000"

In [ ]:
# Turned into a datetime64
start = np.datetime64("2017-01-01T00:00:00.000000000").astype("datetime64[ns]")
finish = np.datetime64("2019-01-01T00:00:00.000000000").astype("datetime64[Y]")
print(f"{finish - start} between {start} and {finish}")
finish - start

In [ ]:
# But note, not leap-second aware!
epoch_start = np.datetime64(0 , "ns")
now = np.datetime64('now', "ns") 
print(epoch_start, now)

In [ ]:
# Arithmetic using timedelta64
one_second = np.timedelta64(1, 's')

time_j2000 = np.datetime64("2000-01-01T12:00:00.0000")
time = np.datetime64("2024-01-01T12:00:00.0000")

dt = time - time_j2000
elapsed_seconds = int(dt / one_second)
print(f"{elapsed_seconds} seconds between {time_j2000} and {time}")
print(f"Note: {int(elapsed_seconds % (365.25 * 86400))} leapseconds")  

In [ ]:
# Make an array of times from start to finish
np.arange(
    start,
    finish,
    step=np.timedelta64(500, 'ms'), #step
    dtype="datetime64[ns]" # we want ns precision
)[:2]  # show the first two only

But `np.linspace` is probably best avoided, as it might give you a headache sometime

In [ ]:
# OK, since linspace in this case returned an array of ints
a = np.linspace(0, 1000, 64) * np.timedelta64(1,"ms") + np.datetime64("2024-01-01T00:00:00.000000000")
print(f"a has shape {a.shape}, ranging from {a[0]} to {a[-1]} with step = {a[1] - a[0]}")

# Not ok, since we're trying to multiply our timedeltas by something less than one
b = np.linspace(0., 1., 64) * np.timedelta64(1,"s") + np.datetime64("2024-01-01T00:00:00.000000000")
print(f"b has shape {b.shape}, ranging from {b[0]} to {b[-1]} with step = {b[1] - b[0]}")

In [ ]:
# To ensure e.g. nanosecond precision, timedelta64s can't be multiplied by something <1.
print(np.linspace(0., 1., 4) * np.timedelta64(1,"s")) 
print(np.linspace(0., 1., 4) *1e9* np.timedelta64(1,"ns"))

---
## Matplotlib
Plotting library, heavily inspired by matlab (or, whatever matlab was like >>10 years ago)


https://matplotlib.org/stable/gallery/index.html

In [ ]:
# For interactive use inside a notebook:
%matplotlib widget 
import matplotlib.pyplot as plt  # The most matlab-like interface to matplotlib

In [ ]:
fig = plt.figure()
plt.plot(np.arange(10), marker = 'x', linestyle=":", color='yellow')
plt.plot(np.arange(10), np.random.randn(10,3) + 12, linestyle="--")
plt.pcolormesh(np.random.randn(16*16).reshape(16,16))
plt.colorbar().set_label("Noise")
plt.ylabel(r"$\lambda_x$")
plt.xlabel("X")

# plt.savefig("something.pdf")
# plt.savefig("something.png", dpi=300)

In [ ]:
# A more space-physics example

from matplotlib.dates import ConciseDateFormatter, AutoDateLocator

tarr = np.arange(
    "2017-01-01T00:00:00.000",
    "2017-01-01T00:10:00.000",
    step=np.timedelta64(500, 'ms'), #step
    dtype="datetime64[ns]" # we want ns precision
)

dt = (tarr - tarr[0])/np.timedelta64(1,"s")


# Panel size ratios
ratios = [0.5, 2., 4., 1., 1.]

# Make a figure with some subplots, which have a shared x-axis.
fig, axs = plt.subplots(
    len(ratios), 1, 
    sharex=True, 
    figsize=(12,6), 
    gridspec_kw=dict(height_ratios=ratios)
)

for ax in axs:
    plt.sca(ax)    
    plt.plot(tarr, np.sin(2. * np.pi * 0.1 * dt))

# To provide control over formatting dates on an axis, we can use these locator/formatter classes:
loc = AutoDateLocator()
axs[-1].xaxis.set_major_locator(loc)
axs[-1].xaxis.set_major_formatter(ConciseDateFormatter(loc))

plt.tight_layout()


## Anatomy of a figure
This is a useful map of the naming of different components and some functions to modify them
![Anatomy of a figure](https://matplotlib.org/stable/_images/sphx_glr_anatomy_001_2_00x.png)

---
## SciPy
A comprehensive package for data analysis...
https://docs.scipy.org/doc/scipy/tutorial/index.html

In [ ]:
from scipy.io import loadmat # To handle .mat files from matlab, for example
from scipy.signal import welch

In [ ]:
#  An example, putting all the above together:

# Generate a sample signal (sine wave with noise)
fs = 1000  # Sampling frequency
t = np.arange(0, 10, 1/fs)  # Time vector from 0 to 10 seconds
f1 = 50  # Frequency of the sine wave
signal = np.sin(2*np.pi*f1*t) + 0.5*np.random.randn(len(t))  # Signal with added noise

# Compute the Power Spectral Density (PSD) using Welch's method
frequencies, psd = welch(signal, fs=fs, nperseg=256)

# Plot the PSD
plt.figure(figsize=(10, 6))
plt.semilogy(frequencies, psd)
plt.title('Power Spectral Density (PSD) using Welch\'s Method')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power/Frequency (dB/Hz)')
plt.grid(True)
plt.show()